<a href="https://colab.research.google.com/github/Rlackdals981010/ai_Study/blob/master/treeEnsemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#트리의 앙상블
#대체로 성능이 좋은 알고리즘 - 랜덤 포레스트
#CSV파일에 가지런히 정리되어 있는 데이터 - 정형 데이터
##어떤 구조로 되어있는 데이터 <- 프로그래머가 다루는 대부분의 데이터, 정형 데이터를 다루는 가장 뛰어난 알고리즘 : 앙상블 학습 (대부분 결정 트리 기반)
#비정형 데이터 : 데이터 베이스나 엑셀로 표현하기 어려운 데이터(텍스트 데이터 (책의 글), 디지털카메라로 찍은 사진, 디지털 음악 등), 비정형 데이터를 다루는 알고리즘 : 신경망 알고리즘

In [12]:
#랜덤 포레스트 : 앙상블 학습의 대표, 앙상블 학습을 적용할 경우 가장 먼저 랜덤 포레스트를 시도해라
##결정 트리를 랜덤하게 만들어 결정 트리의 군집(숲)을 만들고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

#먼저 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만든다.
##데이터 제작 방법 : 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터 제작. (샘플 중복 추출 가능)
###예) 1000개 가방에서 1000개씩 샘플 추출 -> 뽑은 가방을 다시 후보에 반납 : 부트스트랩 샘플 (bootstrap sample)
####기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.
##각 노드 분할시 전체 특성 중 일부 특성을 무작위로 고른 다음 이 중 최선의 분할을 찾음
###분류 모델인 RandomForestClassifier은 기본적으로 전체 특성 개수의 제곱근만큼의 특성 선택 (분류 : 샘플을 몇 개의 클래스 중 하나로 분류)
####즉, 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택
###회귀 모델인 RandomForestRegressor은 전체 특성 사용 (회귀 : 임의의 값을 예측)

#랜덤 포레스트는 샘플과 타겟을 랜덤으로 선택해서 훈련세트에 과대적합되는 것을 막고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
score = cross_validate(rf, train_input, train_target,return_train_score=True, n_jobs=-1)
print(np.mean(score['train_score']),np.mean(score['test_score']))
#근데 결과는 과대적합이 나왔다..

0.9973541965122431 0.8905151032797809


In [13]:
#랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.
rf.fit(train_input,train_target)
print(rf.feature_importances_)

#결정 트리에선 [0.12345626 0.86862934 0.0079144]
#두번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH특성의 중요도가 조금 상승했다.

[0.23167441 0.50039841 0.26792718]


In [14]:
#랜덤 포레스트의 기능에는 자체적으로 모델 평가가 가능하다
#부트 스트랩에 사용되지 않은 샘플 : OOB샘플을 이용해서 모델 평가가 가능하다
rf = RandomForestClassifier(oob_score=True, n_jobs=-1,random_state=42) #oob_score=True로 평가 점수 보는거임
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


In [18]:
#엑스트라 트리
#랜덤 포레스트와 비슷하게 동작. 기본적으로 100개의 결정 트리 훈련
##부트 스트랩 샘플 사용 X -> 결정 트리 제작시 전체 훈련 세트 사용
###분할 방법도 무작위임. 제일 좋은 분할 안찾음
####DecisionTreeClassifier의 splitter = 'random'이 엑스트라 트리가 사용하는 결정트리
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et,train_input,train_target, return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))
#점수는 좀 낮은데 빠른 계산 속도가 장점임
##랜덤 포레스트처럼 특성 중요도를 제공함
et.fit(train_input,train_target)
print(et.feature_importances_)
#회귀버전은 ExtraTreesRegressor

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


In [19]:
#그레이디언트 부스팅
##깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블
###기본적으로 깊이가 3인 결정트리를 100개 사용한다. -> 깊이가 얕아서 과대적합에 강하고 일반적으로 높은 일반화 성능

#경사 하강법을 사용하여 트리를 앙상블에 추가. (분류에서는 로지스틱 손실 함수, 회귀에서는 평균 제곱 오차 함수)
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015
